# Part 1 of 2: Processing an HTML file

One of the richest sources of information is [the Web](http://www.computerhistory.org/revolution/networking/19/314)! In this notebook, we ask you to use string processing and regular expressions to mine a web page, which is stored in HTML format.

> **Note 0.** The exercises below involve processing of HTML files. However, you don't need to know anything specific about HTML; you can solve (and we have solved) all of these exercises assuming only that the data is a semi-structured string, amenable to simple string manipulation and regular expression processing techniques. In a future notebook, you'll see a different method that employs the [Beautiful Soup module](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).
>
> **Note 1.** Following Note 0, there are some outspoken people who believe you should never use regular expressions on HTML. Your instructor finds these arguments to be overly pedantic. For an entertaining take on the subject, see [this blog post](https://blog.codinghorror.com/parsing-html-the-cthulhu-way/).
>
> **Note 2.** The data below is a snapshot from an older version of the Yelp! site. Therefore, you should complete the exercises using the data we've provided, rather than downloading a copy directly from Yelp!.

**The data: Yelp! reviews.** The data you will work with is a snapshot of a recent search on the [Yelp! site](https://yelp.com) for the best fried chicken restaurants in Atlanta. That snapshot is hosted here: https://cse6040.gatech.edu/datasets/yelp-example

If you go ahead and open that site, you'll see that it contains a ranked list of places:

![Top 10 Fried Chicken Spots in ATL as of September 12, 2017](https://cse6040.gatech.edu/datasets/yelp-example/ranked-list-snapshot.png)

**Your task.** In this part of this assignment, we'd like you to write some code to extract this list.

## Getting the data

First things first: you need an HTML file. The following Python code opens a copy of the sample Yelp! page from above.

In [1]:
import hashlib

with open('yelp.htm', 'r', encoding='utf-8') as f:
    yelp_html = f.read().encode(encoding='utf-8')
    checksum = hashlib.md5(yelp_html).hexdigest()
    assert checksum == "4a74a0ee9cefee773e76a22a52d45a8e", "Downloaded file has incorrect checksum!"
    
print("'yelp.htm' is ready!")

'yelp.htm' is ready!


**Viewing the raw HTML in your web browser.** The file you just downloaded is the raw HTML version of the data described previously. Before moving on, you should go back to that site and use your web browser to view the HTML source for the web page. Do that now to get an idea of what is in that file.

> If you don't know how to view the page source in your browser, try the instructions on [this site](http://www.wikihow.com/View-Source-Code).

**Reading the HTML file into a Python string.** Let's also open the file in Python and read its contents into a string named, `yelp_html`.

In [2]:
with open('yelp.htm', 'r', encoding='utf-8') as yelp_file:
    yelp_html = yelp_file.read()
    
# Print first few hundred characters of this string:
print("*** type(yelp_html) == {} ***".format(type(yelp_html)))
n = 1000
print("*** Contents (first {} characters) ***\n{} ...".format(n, yelp_html[:n]))

*** type(yelp_html) == <class 'str'> ***
*** Contents (first 1000 characters) ***
<!DOCTYPE html>
<!-- saved from url=(0079)https://www.yelp.com/search?find_desc=fried+chicken&find_loc=Atlanta%2C+GA&ns=1 -->
<html xmlns:fb="http://www.facebook.com/2008/fbml" class="js gr__yelp_com" lang="en"><!--<![endif]--><head data-component-bound="true"><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><link type="text/css" rel="stylesheet" href="./Best Fried chicken in Atlanta, GA - Yelp_files/css"><style type="text/css">.gm-style .gm-style-cc span,.gm-style .gm-style-cc a,.gm-style .gm-style-mtc div{font-size:10px}
</style><style type="text/css">@media print {  .gm-style .gmnoprint, .gmnoprint {    display:none  }}@media screen {  .gm-style .gmnoscreen, .gmnoscreen {    display:none  }}</style><style type="text/css">.gm-style-pbc{transition:opacity ease-in-out;background-color:rgba(0,0,0,0.45);text-align:center}.gm-style-pbt{font-size:22px;color:white;font-family:Roboto,Arial,san

Oy, what a mess! It will be great to have some code read and process the information contained within this file.

## Exercise (5 points): Extracting the ranking

Write some Python code to create a variable named `rankings`, which is a list of dictionaries set up as follows:

* `rankings[i]` is a dictionary corresponding to the restaurant whose rank is `i+1`. For example, from the screenshot above, `rankings[0]` should be a dictionary with information about Gus's World Famous Fried Chicken.
* Each dictionary, `rankings[i]`, should have these keys:
    * `rankings[i]['name']`: The name of the restaurant, a string.
    * `rankings[i]['stars']`: The star rating, as a string, e.g., `'4.5'`, `'4.0'`
    * `rankings[i]['numrevs']`: The number of reviews, as an **integer.**
    * `rankings[i]['price']`: The price range, as dollar signs, e.g., `'$'`, `'$$'`, `'$$$'`, or `'$$$$'`.
    
Of course, since the current topic is regular expressions, you might try to apply them (possibly combined with other string manipulation methods) find the particular patterns that yield the desired information.

In [7]:
import re

In [8]:
yelp_html_sp = yelp_html.split('After visiting', 1)
yelp_html_1 = yelp_html_sp[1]
yelp_html_sp2 = yelp_html_1.split('food court', 1)
yelp_final = yelp_html_sp2[0]
#print(yelp_final)

In [9]:
#<span class="indexed-biz-name">1.
rankRegex = re.compile('"indexed-biz-name">(?P<rank>\d*)')
mo_rank = (rankRegex.findall(yelp_final))
print(mo_rank)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']


In [10]:
#<span>Gus’s World Famous Fried Chicken</span></a>
nameRegex = re.compile('><span>(?P<name>[\s\S]*?)</span></a>')
mo_name = (nameRegex.findall(yelp_final))
print(mo_name)

['Gus’s World Famous Fried Chicken', 'South City Kitchen - Midtown', 'Mary Mac’s Tea Room', 'Busy Bee Cafe', 'Richards’ Southern Fried', 'Greens &amp; Gravy', 'Colonnade Restaurant', 'South City Kitchen Buckhead', 'Poor Calvin’s', 'Rock’s Chicken &amp; Fries']


In [11]:
#<div class="i-stars i-stars--regular-4 rating-large" title="4.0 star rating">
ratingRegex = re.compile(r'title\="(?P<rating>\d.\d)\sstar\srating">')
mo_rating = ratingRegex.findall(yelp_final)
print(mo_rating)

['4.0', '4.5', '4.0', '4.0', '4.0', '3.5', '4.0', '4.5', '4.5', '4.0']


In [12]:
 #<span class="review-count rating-qualifier">
 #           549 reviews
 #   </span>
reviewsRegex = re.compile(r'(?P<reviews>\d*)\sreviews')
mo_reviews = (reviewsRegex.findall(yelp_final))
for i in range(0, len(mo_reviews)):
    mo_reviews[i] = int(mo_reviews[i])
print(mo_reviews)

[549, 1777, 2241, 481, 108, 93, 350, 248, 1558, 67]


In [13]:
#<span class="business-attribute price-range">$$</span>
priceRegex = re.compile(r'<span class="business-attribute price-range">(?P<price>[\s\S]*?)</span>')
mo_rank = (priceRegex.findall(yelp_final))
print(mo_rank)

['$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$$', '$']


In [14]:
#order --> rank, name, rating, reviews, price-range
#finalRegex = re.compile('<span\sclass="indexed-biz-name">(?P<rank>\d*)><span>(?P<name>[\s\S]*?)</span></a>title\="(?P<stars>\d.\d)\sstar\srating"> (?P<numrevs>\d*)\sreviews[s\S\]<span class="business-attribute price-range">(?P<price>[\s\S]*?)</span>')

In [15]:

yelp_html_sp = yelp_html.split('After visiting', 1)
yelp_html_1 = yelp_html_sp[1]
yelp_html_sp2 = yelp_html_1.split('food court', 1)
yelp_final = yelp_html_sp2[0]

nameRegex = re.compile('><span>(?P<name>[\s\S]*?)</span></a>')
mo_name = (nameRegex.findall(yelp_final))

ratingRegex = re.compile(r'title\="(?P<rating>\d.\d)\sstar\srating">')
mo_rating = ratingRegex.findall(yelp_final)

reviewsRegex = re.compile(r'(?P<reviews>\d*)\sreviews')
mo_reviews = (reviewsRegex.findall(yelp_final))
for i in range(0, len(mo_reviews)):
    mo_reviews[i] = int(mo_reviews[i])

priceRegex = re.compile(r'<span class="business-attribute price-range">(?P<price>[\s\S]*?)</span>')
mo_rank = (priceRegex.findall(yelp_final))

final_list = list(zip(mo_name, mo_rating, mo_reviews, mo_rank))

In [16]:
print(final_list)

[('Gus’s World Famous Fried Chicken', '4.0', 549, '$$'), ('South City Kitchen - Midtown', '4.5', 1777, '$$'), ('Mary Mac’s Tea Room', '4.0', 2241, '$$'), ('Busy Bee Cafe', '4.0', 481, '$$'), ('Richards’ Southern Fried', '4.0', 108, '$$'), ('Greens &amp; Gravy', '3.5', 93, '$$'), ('Colonnade Restaurant', '4.0', 350, '$$'), ('South City Kitchen Buckhead', '4.5', 248, '$$'), ('Poor Calvin’s', '4.5', 1558, '$$'), ('Rock’s Chicken &amp; Fries', '4.0', 67, '$')]


In [17]:
rankings = [{'name': r[0], 'stars':r[1], 'numrevs':r[2], 'price':r[3]} for r in final_list]

In [18]:
print(rankings)

[{'name': 'Gus’s World Famous Fried Chicken', 'stars': '4.0', 'numrevs': 549, 'price': '$$'}, {'name': 'South City Kitchen - Midtown', 'stars': '4.5', 'numrevs': 1777, 'price': '$$'}, {'name': 'Mary Mac’s Tea Room', 'stars': '4.0', 'numrevs': 2241, 'price': '$$'}, {'name': 'Busy Bee Cafe', 'stars': '4.0', 'numrevs': 481, 'price': '$$'}, {'name': 'Richards’ Southern Fried', 'stars': '4.0', 'numrevs': 108, 'price': '$$'}, {'name': 'Greens &amp; Gravy', 'stars': '3.5', 'numrevs': 93, 'price': '$$'}, {'name': 'Colonnade Restaurant', 'stars': '4.0', 'numrevs': 350, 'price': '$$'}, {'name': 'South City Kitchen Buckhead', 'stars': '4.5', 'numrevs': 248, 'price': '$$'}, {'name': 'Poor Calvin’s', 'stars': '4.5', 'numrevs': 1558, 'price': '$$'}, {'name': 'Rock’s Chicken &amp; Fries', 'stars': '4.0', 'numrevs': 67, 'price': '$'}]


In [ ]:
########################

In [19]:
###
### YOUR CODE HERE
###
import re

yelp_html_sp = yelp_html.split('After visiting', 1)
yelp_html_1 = yelp_html_sp[1]
yelp_html_sp2 = yelp_html_1.split('food court', 1)
yelp_final = yelp_html_sp2[0]

nameRegex = re.compile('><span>(?P<name>[\s\S]*?)</span></a>')
mo_name = (nameRegex.findall(yelp_final))

ratingRegex = re.compile(r'title\="(?P<rating>\d.\d)\sstar\srating">')
mo_rating = ratingRegex.findall(yelp_final)

reviewsRegex = re.compile(r'(?P<reviews>\d*)\sreviews')
mo_reviews = (reviewsRegex.findall(yelp_final))
for i in range(0, len(mo_reviews)):
    mo_reviews[i] = int(mo_reviews[i])

priceRegex = re.compile(r'<span class="business-attribute price-range">(?P<price>[\s\S]*?)</span>')
mo_rank = (priceRegex.findall(yelp_final))

final_list = list(zip(mo_name, mo_rating, mo_reviews, mo_rank))

#name, stars, numrevs, price

rankings = [{'name': r[0], 'stars':r[1], 'numrevs':r[2], 'price':r[3]} for r in final_list]

#print(final_list)
#print(final_list[0])
#print(final_list[1])
#print(final_list[2])
#print(final_list[3])
print(rankings[0])
print(rankings[0]['name'])
print(type(rankings))
print(type(rankings[1]))

{'name': 'Gus’s World Famous Fried Chicken', 'stars': '4.0', 'numrevs': 549, 'price': '$$'}
Gus’s World Famous Fried Chicken
<class 'list'>
<class 'dict'>


In [20]:
# Test cell: `rankings_test`

assert type(rankings) is list, "`rankings` must be a list"
assert all([type(r) is dict for r in rankings]), "All `rankings[i]` must be dictionaries"

print("=== Rankings ===")
for i, r in enumerate(rankings):
    print("{}. {} ({}): {} stars based on {} reviews".format(i+1,
                                                             r['name'],
                                                             r['price'],
                                                             r['stars'],
                                                             r['numrevs']))

assert rankings[0] == {'numrevs': 549, 'name': 'Gus’s World Famous Fried Chicken', 'stars': '4.0', 'price': '$$'} \
       or rankings[0] == {'numrevs': 549, 'name': 'Gus&#39;s World Famous Fried Chicken', 'stars': '4.0', 'price': '$$'}
assert rankings[1] == {'numrevs': 1777, 'name': 'South City Kitchen - Midtown', 'stars': '4.5', 'price': '$$'}
assert rankings[2] == {'numrevs': 2241, 'name': 'Mary Mac’s Tea Room', 'stars': '4.0', 'price': '$$'} \
       or rankings[2] == {'numrevs': 2241, 'name': 'Mary Mac&#39;s Tea Room', 'stars': '4.0', 'price': '$$'}
assert rankings[3] == {'numrevs': 481, 'name': 'Busy Bee Cafe', 'stars': '4.0', 'price': '$$'}
assert rankings[4] == {'numrevs': 108, 'name': 'Richards’ Southern Fried', 'stars': '4.0', 'price': '$$'} \
       or rankings[4] == {'numrevs': 108, 'name': 'Richards&#39; Southern Fried', 'stars': '4.0', 'price': '$$'}
assert rankings[5] == {'numrevs': 93, 'name': 'Greens &amp; Gravy', 'stars': '3.5', 'price': '$$'} \
       or rankings[5] == {'numrevs': 93, 'name': 'Greens & Gravy', 'stars': '3.5', 'price': '$$'}
assert rankings[6] == {'numrevs': 350, 'name': 'Colonnade Restaurant', 'stars': '4.0', 'price': '$$'}
assert rankings[7] == {'numrevs': 248, 'name': 'South City Kitchen Buckhead', 'stars': '4.5', 'price': '$$'}
assert rankings[8] == {'numrevs': 1558, 'name': 'Poor Calvin’s', 'stars': '4.5', 'price': '$$'} \
       or rankings[8] == {'numrevs': 1558, 'name': 'Poor Calvin&#39;s', 'stars': '4.5', 'price': '$$'}
assert rankings[9] == {'numrevs': 67, 'name': 'Rock’s Chicken &amp; Fries', 'stars': '4.0', 'price': '$'} \
       or rankings[9] == {'numrevs': 67, 'name': 'Rock&#39;s Chicken &amp; Fries', 'stars': '4.0', 'price': '$'} \
       or rankings[9] == {'numrevs': 67, 'name': 'Rock&#39;s Chicken & Fries', 'stars': '4.0', 'price': '$'} \
       or rankings[9] == {'numrevs': 67, 'name': 'Rock’s Chicken & Fries', 'stars': '4.0', 'price': '$'}

print("\n(Passed!)")

=== Rankings ===
1. Gus’s World Famous Fried Chicken ($$): 4.0 stars based on 549 reviews
2. South City Kitchen - Midtown ($$): 4.5 stars based on 1777 reviews
3. Mary Mac’s Tea Room ($$): 4.0 stars based on 2241 reviews
4. Busy Bee Cafe ($$): 4.0 stars based on 481 reviews
5. Richards’ Southern Fried ($$): 4.0 stars based on 108 reviews
6. Greens &amp; Gravy ($$): 3.5 stars based on 93 reviews
7. Colonnade Restaurant ($$): 4.0 stars based on 350 reviews
8. South City Kitchen Buckhead ($$): 4.5 stars based on 248 reviews
9. Poor Calvin’s ($$): 4.5 stars based on 1558 reviews
10. Rock’s Chicken &amp; Fries ($): 4.0 stars based on 67 reviews

(Passed!)


**Fin!** This cell marks the end of Part 1. Don't forget to save, restart and rerun all cells, and submit it. When you are done, proceed to Part 2.